## apa

In [ ]:
import pandas as pd

In [ ]:
## 1、统计有多少条read以及有多少种apa
total_read = 17981462
total_apa = 71829
df = pd.read_csv("/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/3aqtl/nano_merge_overlap.bed", sep="\t", header=None,usecols = [3,9],names = ['readID','apaID'])
print(len(df))
print(len(df)/total_read)
print(len(df['apaID'].unique()))
print(len(df['apaID'].unique())/total_apa)

12274894
0.6826416005550605
48613
0.6767879268819


In [ ]:
## 图 这里还要重新画的！
# APA上游的motif

# 先将位点改成区间
pre = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.0/3aqtl/nano_merge"
res_dir = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.0/3aqtl"
all_read = pd.read_csv(f'{pre}_readend.bed',sep='\t',header=None,names=['chr','start','end','name','score','strand'])
valid_read = pd.read_csv(f'{pre}_overlap.bed',sep='\t',header=None,usecols = [3],names = ['name'])
filtered_read = all_read[all_read['name'].isin(valid_read['name'])]
filtered_read['start'] = np.where(filtered_read['strand'] == '+', filtered_read['start']-40, filtered_read['start'])
filtered_read['end'] = np.where(filtered_read['strand'] == '+', filtered_read['end'], filtered_read['end'] + 40)
filtered_read.to_csv(f'{pre}_sorted_interval.bed',sep='\t',header=False,index=False)


fasta = "/mnt/hpc/home/xuxinran/REF/hg19/hg19.fa"
call(f"awk '$6 == \"+\" {{print > \"{res_dir}/positive_strand.bed\"}} $6 == \"-\" {{print > \"{res_dir}/negative_strand.bed\"}}' {pre}_sorted_interval.bed", shell=True)
call(f'bedtools getfasta -fi {fasta} -bed {res_dir}/positive_strand.bed -name -tab -fo {pre}_check_motif_+.bed',shell=True)
call(f'bedtools getfasta -fi {fasta} -bed {res_dir}/negative_strand.bed -name -tab -fo {pre}_check_motif_-.bed',shell=True)


df_seq_positive = pd.read_csv(f'{pre}_check_motif_+.bed',sep='\t',header=None,names=['name','seq'])
df_seq_negative = pd.read_csv(f'{pre}_check_motif_-.bed',sep='\t',header=None,names=['name','seq'])
df_seq_positive['seq'] = df_seq_positive['seq'].apply(lambda x: x.upper())
df_seq_negative['seq'] = df_seq_negative['seq'].apply(lambda x: x.upper())

def find_motif_and_position(seq, strand, motifs=["AATAAA", "ATTAAA", "AGTAAA", "TATAAA", "CATAAA", "GATAAA", "AAAAAA"], motifs_reverse=["TTTATT", "TTTAAT", "TTTACT", "TTTATA", "TTTATG", "TTTATC", "TTTTTT"]):
    l = motifs if strand == "+" else motifs_reverse
    for motif in l:
        positions = []
        pos = -1
        while True:
            pos = seq.find(motif, pos + 1)
            if pos == -1:
                break
            positions.append(pos)
        if positions:
            idx = -1 if strand == "+" else 0
            return motif, positions[idx]
    return None, None

motif_info_positive = df_seq_positive['seq'].apply(lambda x: find_motif_and_position(x, "+"))
df_seq_positive['motif'] = motif_info_positive.apply(lambda x: x[0])
df_seq_positive['motif_position'] = motif_info_positive.apply(lambda x: x[1])
motif_info_negative = df_seq_negative['seq'].apply(lambda x: find_motif_and_position(x, "-"))
df_seq_negative['motif'] = motif_info_negative.apply(lambda x: x[0])
df_seq_negative['motif_position'] = motif_info_negative.apply(lambda x: x[1])
df_seq_positive['motif_position'] = df_seq_positive['motif_position'].apply(lambda x: x-40 if x is not None else x)
df_seq_negative['motif_position'] = df_seq_negative['motif_position'].apply(lambda x: x*(-1)-6 if x is not None else x)
df_combined = pd.concat([df_seq_positive, df_seq_negative], axis=0)
df_combined.reset_index(drop=True, inplace=True)

# 统计没比对到motif和阴性对照的数量
none_count = df_combined['motif'].isna().sum()
if none_count != 0:
    none_rate = none_count/len(df_combined)
    print(f"None rate: {none_rate:.2%}")
else:
    print("None rate: 0")
AAAAAA_count = len(df_combined[df_combined['motif'].isin(['AAAAAA', 'TTTTTT'])])
if AAAAAA_count != 0:
    AAAAAA_rate = AAAAAA_count / len(df_combined)
    print(f"AAAAAA rate: {AAAAAA_rate:.2%}")
else:
    print("AAAAAA rate: 0")


# 画图
df_combined = df_combined[df_combined['motif'].notna()]
df_combined_AATAAA = df_combined[df_combined['motif'].isin(['AATAAA', 'TTTATT'])]['motif_position']
df_combined_other = df_combined[~df_combined['motif'].isin(['AATAAA', 'TTTATT', 'AAAAAA', 'TTTTTT'])]['motif_position']

# 计算频数
counter1 = Counter(list(df_combined_AATAAA))
counter2 = Counter(list(df_combined_other))

# 获取所有的值
all_values = sorted(set(list(df_combined_AATAAA) + list(df_combined_other)))

# 生成频数列表
freq1 = [counter1.get(x, 0) for x in all_values]
freq2 = [counter2.get(x, 0) for x in all_values]

# 插值以平滑曲线
x_new = np.linspace(min(all_values), max(all_values), 800)  # 生成更多的点进行插值

f1 = interp1d(all_values, freq1, kind='cubic')
f2 = interp1d(all_values, freq2, kind='cubic')

# 绘制平滑的折线图
plt.figure(figsize=(10, 6))
plt.plot(x_new, f1(x_new), color='#845EC2', alpha=0.7, label='AATAAA')
plt.fill_between(x_new, f1(x_new), color='#845EC2', alpha=0.3)
plt.plot(x_new, f2(x_new), color='#2C73D2', alpha=0.7, label='other motif')
plt.fill_between(x_new, f2(x_new), color='#2C73D2', alpha=0.3)


# 添加标签和标题
plt.xlabel('Distance between signal site and PAS site')
plt.legend(loc='upper right')

plt.show()

call(f'rm {pre}_sorted_interval.bed {res_dir}/positive_strand.bed {res_dir}/negative_strand.bed {pre}_check_motif_+.bed {pre}_check_motif_-.bed',shell=True)

## isoform

In [10]:
## 1、统计有多少条read以及有多少种apa
import pickle

total_read = 17981462
total_apa = 217055

with open('/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/irqtl/nano_merge_isoform_read.pkl', 'rb') as f:
    data = pickle.load(f)

print(len(data))
print(len(data)/total_read)
print(len(set(data.values())))
print(len(set(data.values()))/total_apa)

5277957
0.293522128512131
149855
0.6904010504250074


In [ ]:
# isoform 雷达图

import matplotlib.pyplot as plt
import numpy as np

# 数据
labels = ['Base\nLevel', 'Exon Level', 'Intron Level', 'Intron Chain\nLevel', 'Transcript\nLevel', 'Locus\nLevel']
values = [87.4, 92.8, 98.1, 83.9, 69.5, 67.4]

# 为了闭合雷达图，需要在末尾添加第一个值
values += values[:1]

# 角度
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]

# 绘制雷达图
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
ax.fill(angles, values, color='#FF9488', alpha=0.25)
ax.plot(angles, values, color='#FF9488', linewidth=2)

# 设置标签
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)

# 显示图形
plt.title('Levels Radar Chart')
plt.show()

## 成熟度得分

In [14]:
## 1、统计有多少条read以及有多少种apa
total_read = 17981462
use_read = 14571483
df = pd.read_csv("/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/stqtl/nano_merge_read_stability.csv", usecols = ['readID','stability'])

print(use_read/total_read)

# 获取四分位数
q1 = df['stability'].quantile(0.25)  # 第一个四分位数（25%）
q2 = df['stability'].quantile(0.5)   # 第二个四分位数（50%），即中位数
q3 = df['stability'].quantile(0.75)  # 第三个四分位数（75%）

# 打印结果
print("第一个四分位数 (Q1):", q1)
print("中位数 (Q2):", q2)
print("第三个四分位数 (Q3):", q3)

0.8103614155511938
第一个四分位数 (Q1): 1.0
中位数 (Q2): 1.0
第三个四分位数 (Q3): 1.0


In [ ]:
## RNA成熟度得分折线图
df = pd.read_csv('/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.0/stqtl/nano_merge_read_stability.csv',usecols = [4])

bins = [0, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
labels = ['<0.95', '0.95-0.96', '0.96-0.97', '0.97-0.98', '0.98-0.99', '>0.99']

# 将read stability ratio分成几个区间
read_stability_ratio_binned = pd.cut(df['stability'], bins, labels=labels)

# 计算每个bin的频率
bin_counts = read_stability_ratio_binned.value_counts(normalize=True).sort_index() * 100

# 重置索引以便于绘图
bin_counts = bin_counts.reset_index()
bin_counts.columns = ['Read Stability Ratio Bins', 'Percentage']

# 准备平滑数据
x = np.arange(len(bin_counts))
y = bin_counts['Percentage']
xnew = np.linspace(x.min(), x.max(), 300)

# 插值
spl = make_interp_spline(x, y, k=3)  # type: BSpline
y_smooth = spl(xnew)

# 绘制平滑折线图
plt.figure(figsize=(10, 6))
plt.plot(xnew, y_smooth,color='#70AD47', linewidth=3.5, alpha=0.8)
plt.xticks(x, labels)
# plt.xlabel('Read Stability Score Bins')
plt.ylabel('Percentage of Reads per Stability Score(%)', fontsize=14)
plt.show()

## promoter

In [11]:
## 1、统计有多少条read以及有多少种apa
total_read = 17981462
total_apa = 21529

df = pd.read_csv("/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/puqtl/nano_merge_overlap_uniq.bed", sep="\t", header=None,usecols = [3,9],names = ['readID','apaID'])
print(len(df))
print(len(df)/total_read)
print(len(df['apaID'].unique()))
print(len(df['apaID'].unique())/total_apa)

6355849
0.3534667537044541
13667
0.6348181522597427


In [ ]:
# TSS上游的motif


import pandas as pd
import numpy as np
from subprocess import call

work_dir = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.0/tssqtl"
valid_read = pd.read_csv(f'{work_dir}/nano_merge_overlap_uniq.bed', sep='\t', header=None, usecols=[0, 1, 2, 3, 4, 5], names=['chr', 'start', 'end', 'name', 'score', 'strand'])

# 提取TATAbox上游序列（25-40bp）
# 对于正链，从start提取；对于负链，从end提取
UPSTREAM_OFFSET = 40
DOWNSTREAM_OFFSET = 10

valid_read['s_tata'] = np.where(valid_read['strand'] == '+', 
                                valid_read['start'] - UPSTREAM_OFFSET, 
                                valid_read['end'] + DOWNSTREAM_OFFSET)

valid_read['e_tata'] = np.where(valid_read['strand'] == '+', 
                                valid_read['start'] - DOWNSTREAM_OFFSET, 
                                valid_read['end'] + UPSTREAM_OFFSET)
# 保存到BED文件
valid_read[['chr', 's_tata', 'e_tata', 'name', 'score', 'strand']].to_csv(f'{work_dir}/tata.bed', sep='\t', header=False, index=False)

fasta = "/mnt/hpc/home/xuxinran/REF/hg19/hg19.fa"
call(f'bedtools getfasta -fi {fasta} -bed {work_dir}/tata.bed -s -fo {work_dir}/tata.fa', shell=True)
call(f'meme {work_dir}/tata.fa \
  -o {work_dir}/meme_output \
  -dna \
  -mod zoops \
  -nmotifs 3 \
  -minw 6 \
  -maxw 15 \
  -revcomp',shell=True)
